# Funzioni Giamma 

Il materiale sotto riportato contiene tutto ciò che ho sviluppato in completa autonomia e, laddove necessario, le porzioni di codice utili a farlo girare correttamente.
Questo file è stato realizzato per tenere traccia di ciò che ho svolto **senza effettuare il _commit_ su Github**.


***


## Preliminaries

### Packages importing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from matplotlib.patches import Polygon
%matplotlib inline

### Coordinates transformation

In [ ]:
#Cell dimensions
XCELL = 42.
ZCELL = 13.

# X coordinates translation
global_x_shifts = [994.2, 947.4,-267.4,-261.5,]

# Z coordinates translations
local_z_shifts = [z*ZCELL for z in range(0,4)]
global_z_shifts = [823.5, 0, 823.5, 0]

## (A) Wire position

### Function to read events
_(identica a quella presente nel file `Analysis_Calibration.ipynb` del branch Bottaro su Github)_

In [ ]:
def read_data(event):    
    event_number = int(event[0])
    hits_number  = int(event[1])
    if hits_number == 0:
        hit       = [np.nan]
        chamber   = [np.nan]
        layer     = [np.nan]
        xl_local  = [np.nan]
        xr_local  = [np.nan]
        z_local   = [np.nan]
        time      = [np.nan]
        xl_global = [np.nan]
        xr_global = [np.nan]
        z_global  = [np.nan]
    else:
        hit       = np.arange(hits_number)
        chamber   = np.fromiter((event[2+5*i] for i in range(hits_number)), int)
        layer     = np.fromiter((event[3+5*i] for i in range(hits_number)), int)
        xl_local  = np.fromiter((event[4+5*i] for i in range(hits_number)), float)
        xr_local  = np.fromiter((event[5+5*i] for i in range(hits_number)), float)
        z_local   = np.fromiter((local_z_shifts[i-1]+ZCELL/2 for i in layer), float)
        time      = np.fromiter((event[6+5*i] for i in range(hits_number)), float)
        xl_global = np.fromiter((global_x_shifts[i] for i in chamber), float) - xl_local
        xr_global = np.fromiter((global_x_shifts[i] for i in chamber), float) - xr_local
        z_global  = np.fromiter((global_z_shifts[i] for i in chamber), float) + z_local
        
    dataframe = pd.DataFrame(
        { 'EvNumber' : event_number,
          'Hit'      : hit,
          'Chamber'  : chamber,
          'Layer'    : layer,
          'XL_local' : xl_local,
          'XR_local' : xr_local,
          'Z_local'  : z_local,
          'Time'     : time,
          'XL_global': xl_global,
          'XR_global': xr_global,
          'Z_global' : z_global,
        })

        
    #dataframe.set_index('Hit', inplace=True) # set as index the number of the hit 
    return dataframe, event_number, hits_number

### Function for wire positions

_Da qui inizia la parte originale scritta da me_ : path e funzione per posizioni fili locali e globali 

In [ ]:
#my specific path
path = '/Users/gianmarcopompeo/Desktop/Final Project (group6)/data_merged/calibration'

In [ ]:
x_positions_loc = [] 
z_positions_loc = []
x_positions_glob = [] 
z_positions_glob = []

def wire_positions():
    
    #cycle on all files of the folder calibration
    for filename in os.listdir(path):
        file_path = path + '/' + str(filename)
    

        with open(file_path) as f:
            for line in f:
                event = line.split()
                event = [float(i) for i in event]
                #print(event)
                ev, evNumber, hits = read_data(event)

                XL_list_loc = list(ev['XL_local'])
                XR_list_loc = list(ev['XR_local'])
                Z_list_loc  = list(ev['Z_local'])

                XL_list_glob = list(ev['XL_global'])
                XR_list_glob = list(ev['XR_global'])
                Z_list_glob  = list(ev['Z_global'])


                #clean lists from NaN
                XL_list_loc = [x for x in XL_list_loc if ~np.isnan(x)]
                XR_list_loc = [x for x in XR_list_loc if ~np.isnan(x)]
                XL_list_glob = [x for x in XL_list_glob if ~np.isnan(x)]
                XR_list_glob = [x for x in XR_list_glob if ~np.isnan(x)]

                #exclude empty lists - local
                if (len(XL_list_loc)>0 and len(XR_list_loc)>0):
                    for ii in range(len(XL_list_loc)):
                        average=(XL_list_loc[ii]+XR_list_loc[ii])*0.5
                        x_positions_loc.append(average)
                        z_positions_loc.append(Z_list_loc[ii])

                #exclude empty lists - global
                if (len(XL_list_glob)>0 and len(XR_list_glob)>0):
                    for ii in range(len(XL_list_glob)):
                        average=(XL_list_glob[ii]+XR_list_glob[ii])*0.5
                        x_positions_glob.append(average)
                        z_positions_glob.append(Z_list_glob[ii])
                    
    return x_positions_loc, z_positions_loc, x_positions_glob, z_positions_glob

In [ ]:
wire_positions()

In [ ]:
plt.plot(x_positions_glob, z_positions_glob, "b.")

## Integration of the wire feature
Qui sotto la funzione `plot_background` (anch'essa recuperata dallo stesso file di cui sopra) è stata modificata in modo tale da restituire automaticamente anche il plot delle posizioni dei fili in sovrimpressione. Per ottenerli, è naturalmente necessario aver girato la funzione apposita che fornisce i valori di coordinate nelle liste `x_positions` e `z_positions`

In [ ]:
def plot_background():
    # create Pandas DataFrame for the cambers positions
    chamber_position = pd.DataFrame({
    'chamber' : [i for i in range(4)],
    'x_vertices' : [(global_x_shifts[i], global_x_shifts[i] - 720, global_x_shifts[i] - 720, global_x_shifts[i])
                    for i in range(4)],
    'y_vertices' : [(global_z_shifts[i], global_z_shifts[i], global_z_shifts[i] + 52, global_z_shifts[i] + 52)
                    for i in range(4)],
    })
    x_lim = [[-1000, 1000], # global detector
             [    0, 1000], # chamber 0
             [    0, 1000], # chamber 1
             [-1000,    0], # chamber 2
             [-1000,    0]] # chamber 3
    y_lim = [[-100, 1000],  # global detector
             [800 ,  900],  # chamber 0
             [ -25,   75],  # chamber 1
             [ 800,  900],  # chamber 2
             [ -25,   75]]  # chamber 3
    title = ["DETECTOR", "Chamber 0", "Chamber 1", "Chamber 2", "Chamber 3"]
    # create pyplot 'Axes' objects
    ax_global = plt.subplot2grid(gridsize, (0, 0), colspan=2, rowspan=2)
    ax_0 = plt.subplot2grid(gridsize, (2, 1), colspan=1, rowspan=1) # top-right
    ax_1 = plt.subplot2grid(gridsize, (3, 1), colspan=1, rowspan=1) # bottom-right
    ax_2 = plt.subplot2grid(gridsize, (2, 0), colspan=1, rowspan=1) # top-left
    ax_3 = plt.subplot2grid(gridsize, (3, 0), colspan=1, rowspan=1) # bottom-left
    
    axes = [ax_global, ax_0, ax_1, ax_2, ax_3]
    for index, ax in enumerate(axes):
        ax.set_xlim(x_lim[index])
        ax.set_ylim(y_lim[index])
        ax.set_xlabel("x [mm]")
        ax.set_ylabel("z [mm]")
        ax.plot(x_positions_glob, z_positions_glob, marker=".", ls="", color="gray", markersize=1)
        if index == 0: ax.set_title(title[index])
        else: ax.set_title(title[index], pad=-20)
        # plot the 4 chambers in each 'Axes'
        for j in range(4):
            chamber = chamber_position[chamber_position["chamber"] == j]
            ax.fill(chamber["x_vertices"].values[0], chamber["y_vertices"].values[0], color='gray', fill=False)
    return axes

In [ ]:
gridsize = (5, 2)
fig = plt.figure(figsize = (12, 24))
axes = plot_background()

plt.show()

## (B) Slope differences local-global

# EXTRA

In [ ]:

x_positions = [] 
z_positions = []

def wire_positions():
    x_positions_loc = [] 
    z_positions_loc = []
    x_positions_glob = [] 
    z_positions_glob = []

    file_path = '/Users/gianmarcopompeo/Desktop/Final Project (group6)/data_merged/calibration/Run000261.txt'#here to select if one wants local or global
    #wire positions
    local= False

    with open(file_path) as f:
        for line in f:
            event = line.split()
            event = [float(i) for i in event]
            #print(event)
            ev, evNumber, hits = read_data(event)

            if (local):
                XL_list = list(ev['XL_local'])
                XR_list = list(ev['XR_local'])
                Z_list  = list(ev['Z_local'])
            else:
                XL_list = list(ev['XL_global'])
                XR_list = list(ev['XR_global'])
                Z_list  = list(ev['Z_global'])


            #clean lists from NaN
            XL_list = [x for x in XL_list if ~np.isnan(x)]
            XR_list = [x for x in XR_list if ~np.isnan(x)]

            #exclude empty lists 
            if (len(XL_list)>0 and len(XR_list)>0):
                for ii in range(len(XL_list)):
                    average=(XL_list[ii]+XR_list[ii])*0.5
                    x_positions.append(average)
                    z_positions.append(Z_list[ii])

    #print(len(x_positions), len(z_positions))
    
#return x_positions_loc, z_positions_loc, x_positions_glob, z_positions_glob
